<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/summarize-extract-genQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO : OPTIMIZEEEEEE

# https://turbolab.in/abstractive-summarization-using-googles-t5/

#OPTIMIZE
# https://www.youtube.com/watch?v=2glb3BfOSVE

In [ ]:
# DONT FORGET TO CHANGE TO CUDA VERSION
# !pip3 install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip3 install transformers
!pip3 install SentencePiece
!pip3 install git+https://github.com/boudinfl/pke.git

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pke
import string

In [3]:
T5_MODEL_PATH = 't5-large'

model = T5ForConditionalGeneration.from_pretrained(T5_MODEL_PATH)
tokenizer = T5Tokenizer.from_pretrained(T5_MODEL_PATH)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [25]:
def preprocess_bulk_text(text):
  text = text.strip().replace("\n"," ")
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = tokenizer.encode_plus("summarize: " + text, return_tensors='pt', max_length=512, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [34]:
def postprocess_summary(text):
  output = ""

  for x in sent_tokenize(text):
    x = x.capitalize()
    output += " " + x

  return output

In [30]:
def summarize(text):
  input_tokens_ids, attention_mask = preprocess_bulk_text(text)
  summary_encoded = model.generate(input_ids=input_tokens_ids, 
                                   attention_mask=attention_mask,
                                   num_beams=3,
                                   num_return_sequences=1,
                                   no_repeat_ngram_size=2, 
                                   length_penalty=2.0, 
                                   max_length=300, 
                                   early_stopping=True)
  # decode summarized token
  output = tokenizer.decode(summary_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

  return postprocess_summary(output)

In [41]:
def extract_keywords(text, kw_pop):
  ex = pke.unsupervised.MultipartiteRank()
  ex.load_document(text)   

  pos = {'PROPN','NOUN'}
  stoplist = list(string.punctuation) 
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  ex.candidate_selection(pos=pos, stoplist=stoplist)

  ex.candidate_weighting(alpha=1.1, threshold=0.75, method='average')

  kw = ex.get_n_best(n=kw_pop)

  kw_ls = []

  for i in kw:
    kw_ls.append(i[0])

  return kw_ls

In [42]:
def filter_keywords(original, summarized, kw_pop=10):
  orig_ls = extract_keywords(original, kw_pop)
  sum_ls = extract_keywords(summarized, kw_pop)

  orig_ls = set(orig_ls)
  return list(orig_ls.intersection(sum_ls))

In [36]:
full_text = """
Elon Musk has shown again he can influence the digital currency market with just his tweets. After saying that his electric vehicle-making company
Tesla will not accept payments in Bitcoin because of environmental concerns, he tweeted that he was working with developers of Dogecoin to improve
system transaction efficiency. Following the two distinct statements from him, the world's largest cryptocurrency hit a two-month low, while Dogecoin
rallied by about 20 percent. The SpaceX CEO has in recent months often tweeted in support of Dogecoin, but rarely for Bitcoin.  In a recent tweet,
Musk put out a statement from Tesla that it was “concerned” about the rapidly increasing use of fossil fuels for Bitcoin (price in India) mining and
transaction, and hence was suspending vehicle purchases using the cryptocurrency.  A day later he again tweeted saying, “To be clear, I strongly
believe in crypto, but it can't drive a massive increase in fossil fuel use, especially coal”.  It triggered a downward spiral for Bitcoin value but
the cryptocurrency has stabilised since.   A number of Twitter users welcomed Musk's statement. One of them said it's time people started realising
that Dogecoin “is here to stay” and another referred to Musk's previous assertion that crypto could become the world's future currency.
"""

summary = summarize(full_text)
print(summary)

 Elon musk has shown again he can influence the digital currency market with just his tweets. After saying that his electric vehicle-making company tesla will not accept payments in bitcoin because of environmental concerns, elon tweeted that they were working with developers of dogecoin to improve system transaction efficiency. Following the two distinct statements from him, the world's largest cryptocurrency hit a two-month low, while dogecoin rallied by about 20 percent.


In [43]:
filtered_kw = filter_keywords(full_text, summary)

In [44]:
filtered_kw

['elon musk', 'vehicle', 'dogecoin', 'bitcoin']

In [49]:
T5_Q_MODEL_PATH = 'ramsrigouthamg/t5_squad_v1'
question_model = T5ForConditionalGeneration.from_pretrained(T5_Q_MODEL_PATH)
question_tokenizer = T5Tokenizer.from_pretrained(T5_Q_MODEL_PATH)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [46]:
def preprocess_summary(context, answer):
  text = "context: {} answer: {}".format(context, answer)
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = question_tokenizer.encode_plus(text, return_tensors='pt', max_length = 382, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [47]:
def gen_question(context, answer):
  input_tokens_ids, attention_mask = preprocess_summary(context, answer)

  question_encoded = question_model.generate(input_ids=input_tokens_ids, 
                                             attention_mask=attention_mask,
                                             num_beams=5,
                                             no_repeat_ngram_size=2, 
                                             max_length=72, 
                                             early_stopping=True)
  # decode summarized token
  output = question_tokenizer.decode(question_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  output = output.replace("question: ", "")
  output = output.strip()
  return output

In [53]:
for kw in filtered_kw:
  print(kw)
  print(gen_question(summary, kw))

elon musk
Who has shown again he can influence the digital currency market with just his tweets?
vehicle
What does tesla make?
dogecoin
What cryptocurrency did Elon musk work with to improve system transaction efficiency?
bitcoin
What did Elon musk say tesla would not accept payments in?
